In [5]:
!pip install --quiet openai
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [13]:
import pandas as pd, numpy as np, re

RAW_XLSX = "sample_customer_reviews_raw.xlsx"     # already in your folder
RAW_SHEET = "sample_customer_reviews_raw.csv"       # sheet name
df = pd.read_excel(RAW_XLSX, sheet_name=RAW_SHEET)

print("Shape:", df.shape)
df.head(3)        # quick peek

Shape: (100, 10)


,Review ID,Product Name,Rating,Review Content,Timestamp,Customer Email,Product Category,Order Value,Fulfillment Status,Shipping Country
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Cross-group web-enabled encoding,NaN,Pressure let kind degree list top somebody col...,2025-03-04 01:23:13,williamsdarlene@baker.org,Evening Gowns,NaN,Fulfilled,Germany
1,23b8c1e9-3924-46de-beb1-3b9046685257,Down-sized incremental matrix,1.0,Challenge reach throughout team those sing des...,2025-05-10 16:21:32,ismall@may-turner.com,Prom Dresses,197.72,NaN,Australia
2,bd9c66b3-ad3c-4d6d-9a3d-1fa7bc8960a9,Polarized didactic initiative,1.0,Lead soon property.,2025-03-09 11:07:09,shawn02@yahoo.com,Cocktail Dreses,485.10,NaN,Germany


In [14]:
import pandas as pd
import numpy as np
import re, random
from datetime import datetime

# Load CSV (assumes you've already converted the Excel to CSV)
RAW_CSV  = "sample_customer_reviews_raw.csv"
CLEANED_CSV = "sample_customer_reviews_cleaned.csv"

df = pd.read_csv(RAW_CSV)

# 1️⃣ Normalize timestamps → ISO-8601
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce") \
                     .dt.strftime("%Y-%m-%dT%H:%M:%S")

# 2️⃣ Fix typos in Product Category & Fulfillment Status
cat_map = {
    r"(?i)promdrsses?": "Prom Dresses",
    r"(?i)cocktail dreses?": "Cocktail Dresses"
}
stat_map = {
    r"(?i)delaye?": "Delayed",
    r"(?i)fufilled": "Fulfilled"
}
for patt, repl in cat_map.items():
    df["Product Category"] = df["Product Category"].str.replace(patt, repl, regex=True)
for patt, repl in stat_map.items():
    df["Fulfillment Status"] = df["Fulfillment Status"].str.replace(patt, repl, regex=True)

# 3️⃣ Cast numeric columns
df["Rating"]      = pd.to_numeric(df["Rating"], errors="coerce")
df["Order Value"] = pd.to_numeric(df["Order Value"], errors="coerce")

# 4️⃣ Impute missing ratings with the median
median_rating = df["Rating"].median()
df["Rating"] = df["Rating"].fillna(median_rating)

# 5️⃣ Generate Review Content for missing entries
def generate_review_based_on_rating(rating):
    if rating <= 2:
        return random.choice([
            "Very disappointed with the quality.",
            "Product arrived late and damaged.",
            "Not worth the price.",
            "Won't recommend to anyone."
        ])
    elif rating >= 4:
        return random.choice([
            "Excellent product, very satisfied!",
            "Loved the design and quality.",
            "Will definitely order again.",
            "Highly recommend this item!"
        ])
    else:
        return random.choice([
            "It's okay, not great but not bad.",
            "Average experience.",
            "Product was just fine.",
            "Neutral opinion about this one."
        ])

df["Review Content"] = df.apply(
    lambda row: generate_review_based_on_rating(row["Rating"])
    if pd.isna(row["Review Content"]) else row["Review Content"],
    axis=1
)



In [15]:

missing = df[["Product Category", "Fulfillment Status", "Order Value"]].isna().sum()
print("Missing values per column:\n", missing)


print("\nUnique statuses BEFORE cleaning:\n", df["Fulfillment Status"].unique())


non_numeric = df[pd.to_numeric(df["Order Value"], errors="coerce").isna()]["Order Value"]
print("\nProblematic 'Order Value' cells:\n", non_numeric.unique())


Missing values per column:
 Product Category       7
Fulfillment Status    19
Order Value            3
dtype: int64

Unique statuses BEFORE cleaning:
 ['Fulfilled' nan 'Delayedd' 'Cancelled' 'Delayed' 'Returned']

Problematic 'Order Value' cells:
 [nan]


In [16]:
# 3-A  ----------  Product Category  ----------
# 1) fix obvious typos first (PromDrsses → Prom Dresses, Cocktail Dreses → Cocktail Dresses)
cat_fixes = {r"(?i)promdrsses?": "Prom Dresses",
             r"(?i)cocktail dreses?": "Cocktail Dresses"}
for patt, repl in cat_fixes.items():
    df["Product Category"] = df["Product Category"].str.replace(patt, repl, regex=True)

# 2) fill still-missing categories
# Strategy: if Product Name contains a keyword, infer category; otherwise mark as 'Unknown'
keywords = {"prom": "Prom Dresses",
            "cocktail": "Cocktail Dresses",
            "evening": "Evening Gowns",
            "bridesmaid": "Bridesmaid Dresses",
            "plus": "Plus Size Dresses"}

def infer_cat(row):
    if pd.notna(row["Product Category"]):
        return row["Product Category"]
    name = str(row["Product Name"]).lower()
    for k, v in keywords.items():
        if k in name:
            return v
    return "Unknown"

df["Product Category"] = df.apply(infer_cat, axis=1)

# 3-B  ----------  Fulfillment Status ----------
status_fixes = {r"(?i)fufilled": "Fulfilled",
                r"(?i)delaye?":  "Delayed"}
for patt, repl in status_fixes.items():
    df["Fulfillment Status"] = df["Fulfillment Status"].str.replace(patt, repl, regex=True)

df["Fulfillment Status"] = df["Fulfillment Status"].fillna("Unknown")

# 3-C  ----------  Order Value ----------
df["Order Value"] = pd.to_numeric(df["Order Value"], errors="coerce")
median_val = df["Order Value"].median()
df["Order Value"] = df["Order Value"].fillna(median_val)


In [17]:
print("\nMissing values AFTER cleaning:\n",
      df[["Product Category", "Fulfillment Status", "Order Value"]].isna().sum())

print("\nUnique statuses AFTER cleaning:\n", df["Fulfillment Status"].unique()[:10])

df.loc[df["Product Category"] == "Unknown"].head(3)   # sample still-unknown rows


Missing values AFTER cleaning:
 Product Category      0
Fulfillment Status    0
Order Value           0
dtype: int64

Unique statuses AFTER cleaning:
 ['Fulfilled' 'Unknown' 'Delayeddd' 'Cancelled' 'Delayedd' 'Returned']


,Review ID,Product Name,Rating,Review Content,Timestamp,Customer Email,Product Category,Order Value,Fulfillment Status,Shipping Country
12,1a2a73ed-562b-4f79-8374-59eef50bea63,Centralized actuating paradigm,5.0,Pattern try simple let stay or focus early var...,2025-05-16T23:15:29,kathryn50@yahoo.com,Unknown,164.63,Returned,United Kingdom
16,ec1b8ca1-f91e-4d4c-9ff4-9b7889463e85,Right-sized asymmetric service-desk,1.0,Family identify during professional hard.,2025-04-09T18:58:25,catherinerobertson@yahoo.com,Unknown,213.90,Delayedd,United States
28,3eabedcb-baa8-4dd4-88bd-64072bcfbe01,Persevering regional open system,3.0,Service data near until just recognize buildin...,2025-04-01T18:16:08,david86@yahoo.com,Unknown,344.62,Unknown,United Kingdom


In [19]:
df.to_csv("cleaned_customer_reviews.csv", index=False)
print("✅  cleaned_customer_reviews.csv written to disk")

✅  cleaned_customer_reviews.csv written to disk
